In [1]:
#Install Packages
!pip install faiss-cpu
!pip install sentence-transformers

In [1]:
# import necessary libraries
import pandas as pd
pd.set_option('display.max_colwidth', 100)

C:\Users\mh3511\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
df = pd.read_csv("sample_text.csv")
df.shape

(8, 2)

In [3]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


### Step 1 : Create source embeddings for the text column

In [4]:
from sentence_transformers import SentenceTransformer

In [5]:
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\mh3511\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mh3511\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
vectors.shape

(8, 768)

In [7]:
dim = vectors.shape[1]
dim

768

### Step 2 : Build a FAISS Index for vectors

In [8]:
import faiss

index = faiss.IndexFlatL2(dim)

### Step 3 : Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

In [9]:
index.add(vectors)

In [10]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x0000019FCD420E10> >

### Step 4 : Encode search text using same encorder and normalize the output vector

In [11]:
search_query = "I want to buy a polo t-shirt"
# search_query = "looking for places to visit during the holidays"
# search_query = "An apple a day keeps the doctor away"
vec = encoder.encode(search_query)
vec.shape

(768,)

In [12]:
import numpy as np
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

In [67]:
# faiss.normalize_L2(svec)

### Step 5: Search for similar vector in the FAISS index created

In [14]:
distances, I = index.search(svec, k=2)
distances

array([[1.3844839, 1.4039093]], dtype=float32)

In [15]:
I

array([[3, 2]], dtype=int64)

In [16]:
I.tolist()

[[3, 2]]

In [17]:
row_indices = I.tolist()[0]
row_indices

[3, 2]

In [18]:
df.loc[row_indices]

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion


In [19]:
search_query

'I want to buy a polo t-shirt'

You can see that the two results from the dataframe are similar to a search_query